# This Notebook estimates the model

## Settings

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from time import time
import matplotlib.pyplot as plt
from figure_manager import FigureManager
from plots import *

import DynamicTimeAllocationModel

path = 'output/'

fm = FigureManager(path, use_latex=False)

# c++ settings
do_compile = True
threads = 40

# from EconModel import cpptools
# cpptools.setup_nlopt(folder='cppfuncs/', do_print=True)

In [2]:
# setup model
settings = { 
       'T':10,
       'num_A': 100,
       'num_A_pd': 200,
       'num_K': 2,
       'num_power': 21,
       'num_love': 11,
       
       'simT': 10,
       'simN': 1000,

       'num_Ctot': 120,
       'num_marg_u':120,

       'phi_w': 1.0,
       'phi_m': 1.0,
       
       'mu_w': 0.3,
       
       'p_meet': 0.0,

       'threads':threads,
       'num_multistart': 5,
       
       'do_egm': False,
       'interp_method': 'linear',
       'interp_inverse': False,
       
       'precompute_intratemporal': True,
       'centered_gradient': True,
       }

model = DynamicTimeAllocationModel.HouseholdModelClass(par=settings) 
model.link_to_cpp(force_compile=do_compile)

## Empirical Moments to Match

In [3]:
# all moments listed here will be used in estimation. Comment out those you do not want to use.
datamoms = dict()

# Wages
datamoms['wage_level_w'] = 3.634
datamoms['wage_level_m'] = 3.634

datamoms[('wage_growth_w',5)] = 0.0510
datamoms[('wage_growth_m',5)] = 0.0510

datamoms[('wage_growth_w',10)] = 0.0208
datamoms[('wage_growth_m',10)] = 0.0208

datamoms[('wage_growth_w',15)] = 0.0133
datamoms[('wage_growth_m',15)] = 0.0133


## Parameters to estimate

In [4]:
# parameters to estimate
estpars = {

    # Wages
    'mu_w': {'guess':8.14,'lower':7.5,'upper':9.00},
    'mu_m': {'guess':8.14,'lower':7.5,'upper':9.00},
    
    'gamma_w': {'guess':0.13,'lower':0.08,'upper':0.20},
    'gamma_m': {'guess':0.13,'lower':0.08,'upper':0.20},

}


In [5]:
# check bounds
bounds_ok = True
for key in estpars.keys():
    if estpars[key]['guess']<estpars[key]['lower']:
        print(key,' lower',estpars[key]['guess'])
        bounds_ok = False
    
    if estpars[key]['guess']>estpars[key]['upper']:
        print(key,' upper',estpars[key]['guess'])
        bounds_ok = False

if not bounds_ok:
    stop

## setup initial guess 

In [6]:
# check initial guess
theta_init = np.array([estpars[key]['guess'] for key in estpars.keys()])
obj_init = model.obj_func(theta_init,estpars, datamoms, do_print=True)

Parameters:
  mu_w           : 8.1400 (init: 8.1400)
  mu_m           : 8.1400 (init: 8.1400)
  gamma_w        : 0.1300 (init: 0.1300)
  gamma_m        : 0.1300 (init: 0.1300)
Moments:
  wage_level_w             : sim: -0.6931, data: 3.6340
  wage_level_m             : sim: -0.2877, data: 3.6340
  ('wage_growth_w', 5)     : sim: 0.0000, data: 0.0510
  ('wage_growth_m', 5)     : sim: 0.0000, data: 0.0510
  ('wage_growth_w', 10)    : sim: 0.0000, data: 0.0208
  ('wage_growth_m', 10)    : sim: 0.0000, data: 0.0208
  ('wage_growth_w', 15)    : sim: 0.0000, data: 0.0133
  ('wage_growth_m', 15)    : sim: 0.0000, data: 0.0133
Objective function value: 34.1102 (penalty: 0.0000)
-------------------------------------


## Estimate model

In [ ]:
# Estimate model using nelder-mead algorithm
from scipy.optimize import minimize
do_print = True # Turn off... 
res = minimize(model.obj_func, theta_init, args=(estpars, datamoms,do_print), method='Nelder-Mead',
               options={'xatol': 1e-4, 'fatol': 1e-4, 'disp': True, 'maxiter':5000})


Parameters:
  mu_w           : 8.1400 (init: 8.1400)
  mu_m           : 8.1400 (init: 8.1400)
  gamma_w        : 0.1300 (init: 0.1300)
  gamma_m        : 0.1300 (init: 0.1300)
Moments:
  wage_level_w             : sim: -0.6931, data: 3.6340
  wage_level_m             : sim: -0.2877, data: 3.6340
  ('wage_growth_w', 5)     : sim: 0.0000, data: 0.0510
  ('wage_growth_m', 5)     : sim: 0.0000, data: 0.0510
  ('wage_growth_w', 10)    : sim: 0.0000, data: 0.0208
  ('wage_growth_m', 10)    : sim: 0.0000, data: 0.0208
  ('wage_growth_w', 15)    : sim: 0.0000, data: 0.0133
  ('wage_growth_m', 15)    : sim: 0.0000, data: 0.0133
Objective function value: 34.1102 (penalty: 0.0000)
-------------------------------------
Parameters:
  mu_w           : 8.5470 (init: 8.1400)
  mu_m           : 8.1400 (init: 8.1400)
  gamma_w        : 0.1300 (init: 0.1300)
  gamma_m        : 0.1300 (init: 0.1300)
Moments:
  wage_level_w             : sim: -0.6931, data: 3.6340
  wage_level_m             : sim: -0.2877,